# 오차역전파법

- 수치 미분을 사용해서 가중치 매개변수에 대한 손실 함수의 기울기를 구하는 것은 단순하고 구현하기도 쉽지만 계산 시간이 오래 걸린다.

- 오차역전파법 : 가중치 매개변수의 기울기를 효율적으로 계산

- 이해하는 방법 : 수식 / 계산 그래프

---

# 계산 그래프

- 계산 과정을 그래프로 나타낸 것

- 슈퍼에서 1개에 100원인 사과를 2개 샀을 때 지불금액을 구하시오. 단 소비세가 10% 부과된다.

![image](https://user-images.githubusercontent.com/81560908/235011279-8f108446-9556-4968-81a3-d18f7af9b28d.png)

- '사과 개수'와 '소비세'를 변수로 취급해 원 밖에 표기

### 순전파

- 계산을 왼쪽에서 오른쪽으로 진행하는 단계 / 계산 그래프의 출발점부터 종착점으로의 전파

### 역전파

- 오른쪽에서 왼쪽의 전파

- 미분을 계산할 때 중요한 역할을 한다. 

## 계산 그래프 이점

1. 국소적 계산

    - 전체가 아무리 복잡해도 각 노드에서 단순한 계산에 집중하여 문제를 단순화할 수 있다.

![image](https://user-images.githubusercontent.com/81560908/235114096-d29f19ae-066f-490c-88bc-5b94e95e9c99.png)

 
2. 중간 계산 결과를 모두 보관할 수 있다.

3. 순전파와 역전파를 활용해서 각 변수의 미분을 효율적으로 계산할 수 있다. 

---

# 연쇄 법칙

- 국소적 미분을 전달하는 원리 / 계산 그래프 상의 역전파와 같다.

## 계산 그래프의 역전파

![image](https://user-images.githubusercontent.com/81560908/235013305-7ccae75f-ea03-4023-95e5-25b58fe0007f.png)

- 신호 E에 노드의 국소적 미분을 곱한 후 다음 노드로 전달하는 것

- 목표로 하는 미분 값을 효율적으로 구할 수 있다.

## 원리

- 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

<div style="text-align: center;">
    <img src="https://user-images.githubusercontent.com/81560908/235023055-c5186fbe-1938-41a7-8fed-e6d65c7a2e3a.png" style="display: inline-block;">
    <img src="https://user-images.githubusercontent.com/81560908/235023237-3112d68d-ca6f-4e3b-9767-90849f37e539.png" style="display: inline-block;">
</div>

---

# 역전파

## 덧셈 노드의 역전파

z = x +y 

![image](https://user-images.githubusercontent.com/81560908/235023497-40a9f17b-70cd-41c5-82e4-70f01911a3df.png)

## 곱셈 노드의 역전파

z = xy

![image](https://user-images.githubusercontent.com/81560908/235023404-c8344d0c-eeed-44d8-aff4-88c8a61cc449.png)

---

# 단순한 계층 구현하기

## 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):                   # 순전파 시의 입력 값을 유지하기 위해 인스턴스 변수인 x와 y를 초기화
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y
        
        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

![image](https://user-images.githubusercontent.com/81560908/235027211-6ee84ba8-1c9f-4447-ae94-b62384cf2ce9.png)

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층 (layer)
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 (forward)
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("price:", int(price))

price: 220


In [3]:
# 역전파 (backward)

dprice = 1   # 순전파의 출력에 대한 미분
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

dApple: 2.2
dApple_num: 110
dTax: 200


## 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):                   
        pass

    def forward(self, x, y):          
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

![image](https://user-images.githubusercontent.com/81560908/235026999-4cbe1a00-840b-4d06-91b3-60badeb6ee0f.png)

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


---

# 활성화 함수 계층 구현하기

## ReLU 계층

<div style="text-align: center;">
    <img src="https://user-images.githubusercontent.com/81560908/235031149-bbe71c59-ae10-4993-b24c-4d19e1406de2.png" style="display: inline-block;">
    <img src="https://user-images.githubusercontent.com/81560908/235031215-67d926d6-325d-45cd-9ce1-f22cdca73b46.png" style="display: inline-block;">
</div>

- 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘린다. 
- 순전파 때의 입력인 x가 0 이하면 역전파 때는 하류로 신호를 보내지 않는다. (0을 보낸다.)

    ![image](https://user-images.githubusercontent.com/81560908/235032225-9e6525e9-8253-4a63-9e39-c58bf89036d0.png)

In [6]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

## Sigmoid 함수

![image](https://user-images.githubusercontent.com/81560908/235033969-6971bfe1-6c39-47c5-99f5-39adc68d61cd.png)

![image](https://user-images.githubusercontent.com/81560908/235034463-3245cfc2-6a02-4662-9193-07b0e1fda01b.png)

![image](https://user-images.githubusercontent.com/81560908/235034405-19396c40-79f8-441b-86c6-fed6bb05d7fd.png)

- 순전파의 입력 x와 출력 y만으로 계산할 수 있다.

=> 중간 과정을 모두 묶어 단순한 'sigmoid' 노드 하나로 대체할 수 있다.

![image](https://user-images.githubusercontent.com/81560908/235034967-b0c0bbc4-ce6b-4265-a469-cb61d1c6c3dd.png)

- 순전파의 출력(y)만으로 계산할 수 있다.

![image](https://user-images.githubusercontent.com/81560908/235035137-dc82f471-0bf1-41f4-9c41-0628a40fd788.png)

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out             # 순전파의 출력을 out에 보관했다가 역전파 계산 때 사용

        return dx

---

# Affine/Softmax 계층 구현하기

## Affine 계층

- 어파인 변환을 수행하는 처리

- 어파인 변환 (affine transformation) : 신경망의 순전파 때 수행하는 행령의 곱

X : 입력  W : 가중치  B : 편향

Y = np.dot(X, W) + B

Y => 활성화 함수로 변환해 다음 층으로 전파하는 것이 신경망 순전파의 흐름

- 행렬의 곱에서는 대응하는 차원의 원소 수를 일치시키는 것이 핵심이다.

![image](https://user-images.githubusercontent.com/81560908/235036524-b1c7e3ea-3f38-4612-bb8d-399a159a23b1.png)

### 계산 그래프

![image](https://user-images.githubusercontent.com/81560908/235038902-c93cf15c-b0e4-4cc5-823a-ad94f25bfb06.png)

## 배치용 Affine 계층

- 데이터 N개를 묶어 순전파하는 경우

![image](https://user-images.githubusercontent.com/81560908/235039777-c0a9939b-fb03-4a51-9fbb-707fdfbdc20d.png)

In [8]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

## Softmax-with-Loss 계층

- 소프트맥스 계층을 구현할 때, 손실 함수인 교차 엔트로피 오차도 포함해서 구현하는 경우

### Softmax 계층

- 입력 값을 정규화하여 출력한다. (출력의 합이 1이 되도록 변형)

- 추론할 때는 필요없고 신경망을 학습할 때 필요하다.

![image](https://user-images.githubusercontent.com/81560908/235042297-7bb0242e-086d-4a2b-bcfc-692b9bd521ed.png)

In [9]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

---

# 오차역전파법 구현하기

- 지금까지 구현한 계층을 조합해서 신경망 구축

## 오차역전파법 적용한 신경망 구현

In [10]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

import sys
sys.path.append('/path/to/root')


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

## 오차역전파법으로 구한 기울기 검증

수치 미분은 구현하기 쉬워서 버그가 숨어 있기 어려운 반면, 오차역전파법은 구현하기 복잡해서 종종 실수를 한다.

=> 수치 미분의 결과와 오차역전파법의 결과를 비교하여 오차역전파법을 제대로 구현했는지 검증할 수 있다. 

In [11]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:2.0015135274584279e-10
b1:9.41582337813052e-10
W2:6.82173418468518e-08
b2:1.370578232615105e-07
